# Subsystems enrichment analysis 

Performs the subsystems enrichment analysis based on the hypergeometric test:
$$P(x \geq k) = 1 - hypergeom.cdf(k-1, M, n, N)$$

* k: number of diferentially expressed reactions in a subsystem,
* n: number of diferentially expressed reactions in the model,
* N: number of reactions in a subsystem,
* M: number of reactions in the model.



In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 

from scipy.stats import hypergeom
import statsmodels.stats.multitest as multi

Set the folder in which data are stored.

In [28]:
p_vals_folder = "results_enrichment_reactions"
active_reactions_folder = "results_active_reactions_pairs"

results_folder = "results_enrichment_subsystems"

Read the subsystems data

In [29]:
df_subsystems = pd.read_csv("model\\Human-GEM_subsystems.txt", sep=";")
df_subsystems = df_subsystems.rename(columns={'rxn': 'reaction'})
subsystems = df_subsystems.subsystem.unique()

## Hypergeometric test
Go through the results of the reactions' enrichment analyses and use those to perform the subsystems' enrichment analysis.


In [30]:
file_names = [file_name for file_name in os.listdir(p_vals_folder) if file_name.endswith('csv')]
for file in file_names:
    df_enrichment = pd.DataFrame(columns=["subsystem", "p_up", "p_down", "q_up", "q_down", "enrichment"])
    df_enrichment["subsystem"] = subsystems
    
    df = pd.read_csv(p_vals_folder+"\\"+file, sep=",")
    f = open(active_reactions_folder+"\\"+file)
    active_reactions = f.read().strip().split(";")
    f.close()
    
    df = df[df['reaction'].isin(active_reactions)] # keep only reactions that are active in at least one of the models
    
    M = len(df) # number of different reactions in pairs of models
    n_up = sum(df.enrichment == 1) # number of upregulated reactions in models
    n_down = sum(df.enrichment == -1)  # number of downregulated reactions in models
    
    #print("M:",M)
    #print("n_up:",n_up)
    #print("n_down:",n_down)
    
    for subsystem in subsystems:
        subsystem_reactions = df_subsystems.loc[df_subsystems.subsystem == subsystem,'reaction'].values
        df_sub = df[df['reaction'].isin(subsystem_reactions)] # take only the reactions that belong to this subsystem
        
        N = len(df_sub) # number of reactions in a subsystem
        k_up = sum(df_sub.enrichment == 1)# number of upregulated reactions in a subsystem
        k_down = sum(df_sub.enrichment == -1)# number of downregulated reactions in a subsystem
        
        if n_up:         
            p_up = 1 - hypergeom.cdf(k_up-1, M, n_up, N)                
        else:
            p_up = 1.0
        
        if n_down:         
            p_down = 1 - hypergeom.cdf(k_down-1, M, n_down, N)                
        else:
            p_down = 1.0
        
        df_enrichment.loc[df_enrichment["subsystem"] == subsystem, 'p_up'] = p_up
        df_enrichment.loc[df_enrichment["subsystem"] == subsystem, 'p_down'] = p_down
    
    df_enrichment['q_up'] = multi.multipletests(df_enrichment['p_up'], method = 'fdr_bh')[1]
    df_enrichment['q_down'] = multi.multipletests(df_enrichment['p_down'], method = 'fdr_bh')[1]
    
    df_enrichment.loc[(df_enrichment['q_up']<0.05) & (df_enrichment['q_up']<df_enrichment['q_down']),'enrichment'] = 1
    df_enrichment.loc[(df_enrichment['q_down']<0.05) & (df_enrichment['q_down']<=df_enrichment['q_up']),'enrichment'] = -1
    df_enrichment = df_enrichment.fillna(0)

    df_enrichment.to_csv(results_folder+"\\"+file, index=False)
    
